Tournament bracket generation with byes

That's how the tournament and his matches are generated.
- How to advance to the next round? Using  triggers:
  - When creating a new Match, whether is a bye Match or not, if so:
    - Divide the `match_number // 2`, thats the next Match, and check if `match_number % 2 == 0`, if so, then is team A, otherwise is team B.
    - Matches must be created from the final (`match_number = 1`), so we can update a "next Match" that already exists.
  - Updating a Match, resolve for the winner_id and update the next Match using `match_number // 2` and `match_number % 2 == 0`

In [5]:
from typing import Optional, List, Dict, Tuple
from enum import Enum
import math
from dataclasses import dataclass


class TournamentStatus(Enum):
    PENDING = 'pending'
    IN_PROGRESS = 'in_progress'
    COMPLETED = 'completed'


class MatchStatus(Enum):
    PENDING = 'pending'
    COMPLETED = 'completed'


@dataclass
class Team:
    """Represents a tournament team with seeding information."""
    name: str
    seed_score: int


@dataclass
class Match:
    """Represents a single match in a tournament bracket."""
    level: int  # Round of the tournament (0=final, 1=semifinal, etc.)
    match_number: int  # Position in the bracket
    team_a: Optional[Team] = None
    team_b: Optional[Team] = None
    winner: Optional[Team] = None
    is_bye: bool = False
    status: MatchStatus = MatchStatus.PENDING

@dataclass
class Tournament:
    """Represents a single elimination tournament."""
    name: str
    current_level: int = -1
    status: TournamentStatus = TournamentStatus.PENDING


class TournamentGenerator:
    """
    Generates single-elimination tournament brackets with proper seeding and byes.
    
    Implements standard tournament seeding where:
    - Best seeds play against worst seeds in early rounds
    - Byes are given to top seeds when needed
    - Matches are properly ordered in the bracket
    """
    
    @staticmethod
    def _calculate_bracket_size(team_count: int) -> Tuple[int, int]:
        """
        Calculate the required bracket size and number of byes needed.
        
        Args:
            team_count: Number of teams in the tournament
            
        Returns:
            Tuple of (bracket_size, byes_count)
        """
        if team_count < 2:
            raise ValueError("At least two teams are required for a tournament.")
            
        bracket_size = 2 ** math.ceil(math.log2(team_count)) if team_count > 0 else 1
        byes_count = bracket_size - team_count
        return bracket_size, byes_count

    @staticmethod
    def _seed_teams(teams: List[Team]) -> List[Team]:
        """Sort teams by seed score (best seeds first)."""
        return sorted(teams, key=lambda x: x.seed_score, reverse=True)

    @staticmethod
    def _create_initial_matches(
        sorted_teams: List[Team],
        level: int,
        byes_count: int
    ) -> List[Match]:
        """
        Create the initial matches for the first round of the tournament.
        
        Handles both regular matches and byes.
        """
        matches = []
        
        # Create bye matches (automatic wins for top seeds)
        for i in range(byes_count):
            bye_team = sorted_teams[i]
            matches.append(Match(
                level=level,
                match_number=0,  # Temporary, will be set in seed_matches
                team_a=bye_team,
                is_bye=True,
                status=MatchStatus.COMPLETED,
                winner=bye_team
            ))
        
        # Create regular matches for remaining teams
        remaining_teams = sorted_teams[byes_count:]
        for i in range(0, len(remaining_teams), 2):
            team_a = remaining_teams[i]
            team_b = remaining_teams[i+1] if i+1 < len(remaining_teams) else None
            
            matches.append(Match(
                level=level,
                match_number=0,  # Temporary
                team_a=team_a,
                team_b=team_b,
                is_bye=False,
                status=MatchStatus.PENDING
            ))
            
        return matches

    @staticmethod
    def _assign_match_numbers(matches: List[Match], level: int, bracket_size: int) -> List[Match]:
        """
        Assign proper match numbers following tournament seeding patterns.
        
        This ensures matches are ordered correctly in the bracket.
        """
        if not matches:
            return []
            
        if len(matches) == 1:
            matches[0].match_number = 1
            return matches
            
        # Generate the ordered positions for matches in this level
        match_positions = list(range(2**level, 2**(level+1)))
        
        # We'll distribute matches following standard seeding patterns
        ordered_matches = []
        positions_queue = match_positions.copy()
        
        # First match goes to first position, second to last, then fill middle
        if positions_queue:
            matches[0].match_number = positions_queue.pop(0)
            ordered_matches.append(matches[0])
            
        if len(matches) > 1 and positions_queue:
            matches[1].match_number = positions_queue.pop(-1)
            ordered_matches.append(matches[1])
            
        # Distribute remaining matches to middle positions
        for match in matches[2:]:
            if positions_queue:
                middle_index = len(positions_queue) // 2
                match.match_number = positions_queue.pop(middle_index)
                ordered_matches.append(match)
                
        return ordered_matches

    @staticmethod
    def generate_initial_matches(teams: List[Team]) -> List[Match]:
        """
        Generate the first round of matches for the tournament with proper seeding.
        
        Args:
            teams: List of Team objects participating in the tournament
            
        Returns:
            List of properly seeded Match objects for the first round
        """
        if len(teams) < 2:
            raise ValueError("At least two teams are required to generate matches.")
            
        sorted_teams = TournamentGenerator._seed_teams(teams)
        bracket_size, byes_count = TournamentGenerator._calculate_bracket_size(len(teams))
        level = int(math.log2(bracket_size)) - 1
        
        matches = TournamentGenerator._create_initial_matches(sorted_teams, level, byes_count)
        ordered_matches = TournamentGenerator._assign_match_numbers(matches, level, bracket_size)
        
        return ordered_matches

    @staticmethod
    def generate_full_bracket(teams: List[Team]) -> Dict[int, List[Match]]:
        """
        Generate a complete single-elimination tournament bracket.
        
        Args:
            teams: List of Team objects participating in the tournament
            
        Returns:
            Dictionary mapping levels (rounds) to their matches
        """
        initial_matches = TournamentGenerator.generate_initial_matches(teams)
        bracket_size, _ = TournamentGenerator._calculate_bracket_size(len(teams))
        
        matches_by_level = {}
        current_level = int(math.log2(bracket_size)) - 1  # Starting level
        
        # Add initial matches
        matches_by_level[current_level] = initial_matches
        current_level -= 1
        
        # Generate empty matches for future rounds
        while current_level >= 0:
            matches_count = 2**current_level
            matches = []
            
            for match_num in range(1, matches_count + 1):
                matches.append(Match(
                    level=current_level,
                    match_number=match_num,
                    is_bye=False,
                    status=MatchStatus.PENDING
                ))
                
            matches_by_level[current_level] = matches
            current_level -= 1
            
        return matches_by_level

In [6]:
# Create teams (no IDs needed)
teams = [Team(name=f"Team {i}", seed_score=100-i) for i in range(1, 7)]

# Generate full bracket
full_bracket = TournamentGenerator.generate_full_bracket(teams)

# Print the bracket structure
for level, matches in full_bracket.items():
    print(f"\nLevel {level} (Round of {2**level}):")
    for match in matches:
        teams_str = f"{match.team_a.name if match.team_a else 'TBD'} vs {match.team_b.name if match.team_b else 'TBD'}"
        print(f"Match {match.match_number}: {teams_str} (Status: {match.status.value})")


Level 2 (Round of 4):
Match 4: Team 1 vs TBD (Status: completed)
Match 7: Team 2 vs TBD (Status: completed)
Match 6: Team 3 vs Team 4 (Status: pending)
Match 5: Team 5 vs Team 6 (Status: pending)

Level 1 (Round of 2):
Match 1: TBD vs TBD (Status: pending)
Match 2: TBD vs TBD (Status: pending)

Level 0 (Round of 1):
Match 1: TBD vs TBD (Status: pending)
